# Molecular Dynamics 4 

In [1]:
import turtle
import random
import math

In [2]:
random.seed()

In [3]:
window = turtle.Screen()
window.title('Molecular Dynamics 4')
window.clear()

In [4]:
# We will want to store these numbers in a variable
height = window.window_height()
width = window.window_width()

In [5]:
# Number of atoms we want
num_atoms = 30

# This initializes an empty list
atoms = []

# Now use a loop to initialize a new atom
# and do it num_atoms times.
for i in range(num_atoms):
    atoms.append(turtle.Turtle())

In [6]:
# Max velocity we want
max_velocity = 80.0

# Initailize an empty list
vel = []

# Now loop over the number of atoms
for i in range(num_atoms):
    vel.append([random.uniform(-1,1)*max_velocity, random.uniform(-1,1)*max_velocity])

In [7]:
# Lets name our function
def distance(atom1, atom2):
    
    # Inputs are the atoms, which contain
    # (x,y) coordinate pairs.
    # Distance is sqrt((x1-x2)^2 + (y1-y2)^2)
    
    # Lets get the coordinates of atom1
    (x1,y1) = atom1.pos()
    
    # Lets get the coordinates of atom2
    (x2,y2) = atom2.pos()

    # Lets calculate the distance
    d = math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
    # And lets return our distance
    return d
    

In [8]:
# Variables to hold things we want to be constant
atom_radius = 20

# Scaling factor here is so we don't get our 
# initial positions stuck on the edge.
scaling_factor = 0.8

# We need to loop over each atom.
for i in range(num_atoms):
   
    # Draw the atom in the proper shape
    atoms[i].shape('circle')
    atoms[i].shapesize(atom_radius/10.0)
    atoms[i].color((random.random(),random.random(),random.random()))
    if i % 2 == 0:
        atoms[i].color((1,1,1))
    atoms[i].penup()
    atoms[i].goto(random.uniform(-1,1)*width/2.0 * scaling_factor, random.uniform(-1,1)*height/2.0 * scaling_factor)
  
    # Lets make sure this atom isn't on top of another atom
    # Lets get the minimum distance to all other atoms
    d = 1000000.0
    for j in range(0,i):
        d = min(d, distance(atoms[i],atoms[j]))
        print d
    # Check if we're too close
    while d < 2.0*atom_radius:
        
        # Reset d
        d = 1000000.0
        
        # Let us know we're too close
        print "Atom too close. Repositioning."
        print "   (%d,%d) = %f" % (i, j, d)
        # Choose a new random spot and remeasure
        atoms[i].goto(random.uniform(-1,1)*width/2.0 * scaling_factor, 
                      random.uniform(-1,1)*height/2.0 * scaling_factor)
        for j in range(0,i):
            d = min(d, distance(atoms[i],atoms[j]))
            print "      (%d,%d) = %f" % (i,j,d)
        
    # Turtles can be very slow. This is a semi-fix to tell
    # turtles not to update the screen with every change,
    # but rather wait till a set of updates are done and 
    # then update the screen.
    atoms[i].tracer(0,0)
    turtle.update()
    

425.601374301
289.869185946
142.706672289
103.431172985
103.431172985
103.431172985
229.065435656
229.065435656
164.661786781
148.083494923
211.87068721
211.87068721
78.5771552354
78.5771552354
78.5771552354
186.266433182
186.266433182
186.266433182
186.266433182
186.266433182
186.266433182
224.987476374
201.047612001
68.2456403627
68.2456403627
68.2456403627
16.3198418845
16.3198418845
Atom too close. Repositioning.
   (7,6) = 1000000.000000
      (7,0) = 367.433973
      (7,1) = 367.433973
      (7,2) = 367.433973
      (7,3) = 367.433973
      (7,4) = 367.433973
      (7,5) = 367.433973
      (7,6) = 182.899787
207.273789549
207.273789549
207.273789549
207.273789549
207.273789549
207.273789549
40.2513176431
40.2513176431
106.804615592
106.804615592
106.804615592
106.804615592
106.804615592
106.804615592
97.4103844506
97.4103844506
97.4103844506
446.680036348
90.627590396
90.627590396
90.627590396
90.627590396
90.627590396
90.627590396
90.627590396
90.627590396
90.627590396
346.86069

In [9]:
# Lets name our function
def bounce_check(a_list, v_list):
  
    # This will help us not double count
    i = 1

    # We need to check for every atom
    for atom1 in a_list:
        
        # This will help us not double count
        j = i
        
        # And compute the distance to every other atom
        for atom2 in a_list[i:]:
            
            # We will need the distance, so store it
            d = distance(atom1,atom2)
            
            # Check if a bounce should occur
            if d < 2.0*atom_radius:
                print "BOUNCE FOUND: Atom %d hit Atom %d" % (i-1,j)
                # We need positions of atoms
                (x1,y1) = atom1.pos()
                (x2,y2) = atom2.pos()
                
                # A bounce should occur: fix x_velocity
                # First, calculate v1.r
                v1R = v_list[i-1][0]*(x1-x2) + v_list[i-1][1]*(y1-y2) #projection of first atom velocity onto R
                v2R = v_list[j][0]*(x1-x2) + v_list[j][1]*(y1-y2) #projection of 2nd atom velocity onto R
                
                # Update new velocities
                v_list[i-1][0] = v_list[i-1][0] + (v2R-v1R)/d**2 * (x1-x2)
                v_list[i-1][1] = v_list[i-1][1] + (v2R-v1R)/d**2 * (y1-y2)
                v_list[j][0] = v_list[j][0] + (v1R-v2R)/d**2 * (x1-x2)
                v_list[j][1] = v_list[j][1] + (v1R-v2R)/d**2 * (y1-y2)

            # Update second counter
            j = j + 1
            
        # Update counter
        i = i + 1
    return v_list

In [10]:
# The amount of time each iteration moves us forward
dt = 0.05

# Max number of steps we want to take
max_steps = 10000

# Print headers for output
#print " "
#print "   time         ke            pe             e      "
#print " -------    -----------   ------------  ------------"  

for step in range(max_steps):

    # Calculate forces
    #f, pe = forces(atoms)
    
    # For each atom
    for j in range(num_atoms):
    
        # Get the current position, velocity, and force of the atom
        vx,vy = vel[j]    
        #Fx,Fy = f[j]  
        #vx += Fx*dt*0.5
        #vy += Fy*dt*0.5
        x,y = atoms[j].pos()
    
        # Check if moving left or right will put our atom beyond the wall
        if abs(x + dt * vel[j][0]) >= width/2.0 - atom_radius:
        
            # We have moved too far right or left, so flip the x_vel
            vel[j][0] = -vel[j][0]           
    
        # Check if moving up or down will put our atom beyond the wall
        if abs(y + dt * vel[j][1]) >= height/2.0 - atom_radius:
        
            # We have moved too far up or down, so flip the y_vel
            vel[j][1] = -vel[j][1]
        
    
    vel = bounce_check(atoms,vel)
    
    for j in range(num_atoms):
        x,y = atoms[j].pos()
        atoms[j].goto(x + dt*vel[j][0], y + dt*vel[j][1])
        
        
    # make the forces at time t+dt
    #f, potential_energy = forces(atoms)
            
    # finish update of v to time t+dt
    #kinetic_energy = 0.0
    #for i in range(num_atoms):
        #vx,vy = vel[i]
        #Fx,Fy = f[i]
        #vx += Fx*dt*0.5
        #vy += Fy*dt*0.5
        #vel[i] = [vx, vy]
        #kinetic_energy += 0.5*(vx*vx+vy*vy)
        
        
    # Tell turtles we are done updating and to redraw
    turtle.update()
    
    # Output energies to make sure its working
    #energy = kinetic_energy + potential_energy
    #print "%8.2f   %12.5f   %12.5f  %12.5f  " % (step*dt, kinetic_energy, potential_energy, energy) 
    

BOUNCE FOUND: Atom 3 hit Atom 20
BOUNCE FOUND: Atom 6 hit Atom 8
BOUNCE FOUND: Atom 18 hit Atom 24
BOUNCE FOUND: Atom 2 hit Atom 11
BOUNCE FOUND: Atom 24 hit Atom 26
BOUNCE FOUND: Atom 18 hit Atom 24
BOUNCE FOUND: Atom 7 hit Atom 23
BOUNCE FOUND: Atom 17 hit Atom 25
BOUNCE FOUND: Atom 11 hit Atom 16
BOUNCE FOUND: Atom 14 hit Atom 17
BOUNCE FOUND: Atom 22 hit Atom 27
BOUNCE FOUND: Atom 2 hit Atom 20
BOUNCE FOUND: Atom 11 hit Atom 28
BOUNCE FOUND: Atom 4 hit Atom 10
BOUNCE FOUND: Atom 13 hit Atom 25
BOUNCE FOUND: Atom 4 hit Atom 10
BOUNCE FOUND: Atom 10 hit Atom 28
BOUNCE FOUND: Atom 2 hit Atom 22
BOUNCE FOUND: Atom 4 hit Atom 10
BOUNCE FOUND: Atom 17 hit Atom 25
BOUNCE FOUND: Atom 7 hit Atom 8
BOUNCE FOUND: Atom 11 hit Atom 28
BOUNCE FOUND: Atom 0 hit Atom 27
BOUNCE FOUND: Atom 18 hit Atom 23
BOUNCE FOUND: Atom 4 hit Atom 5
BOUNCE FOUND: Atom 22 hit Atom 27
BOUNCE FOUND: Atom 9 hit Atom 17
BOUNCE FOUND: Atom 9 hit Atom 21
BOUNCE FOUND: Atom 16 hit Atom 19
BOUNCE FOUND: Atom 25 hit Atom 

It works!

Always clean up.

In [11]:
window.bye()